# Assignment 4
In this task you have to scrape all the results from "Extranjero". [Here is link](https://resultadoshistorico.onpe.gob.pe/EG2021/EleccionesPresidenciales/RePres/E). 
You have to **follow the steps as we did in Lecture 7** and get information from all tabs in "Continente", "País" and "Estado".

1. Take the data table TOTAL DE VOTOS.
2. Store the four columns ORGANIZACIONES POLÍTICAS, TOTAL, %VÁLIDOS and %EMITIDOS. Rename these columns to political_parties, total, share_valid, share_emitied, respectively. 
3. Add the columns continent, country, state, ubigeo, and link. These last two items you can extract from the url. 
4. Append  the tables in just one DataFrame and export an excel file and save it in the folder [\_data\_results](https://github.com/alexanderquispe/Diplomado_PUCP/tree/main/Lecture_7/_data_results).
5. The name of your Excel file should be the same as the name of your branches. 

# <a id='outline'> Assignment 4 - Outline </a>
- <a href='#1_Lib'> 1. Libraries required </a>
- <a href='#2_web_page'> 2. Webpage </a>
- <a href='#3_scope'> 3. Scope </a>
- <a href='#4_set_up'> 4. Set Up before loop </a>
- <a href='#5_loop'> 5. Loop for webscraping </a>
- <a href='#6_final_data'> 6. Data processing and storing </a>

## <a id='1_Lib'> 1. Libraries required </a>

In [1]:
#!pip install lxml
#!pip install lxmunidecodel
#!pip install unidecode


import os
import time
import re
import unidecode
import time 


import pandas as pd
import numpy as np


from selenium import webdriver
from selenium.webdriver import ActionChains
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException
from webdriver_manager.chrome import ChromeDriverManager

ERROR: Could not find a version that satisfies the requirement lxmunidecodel
ERROR: No matching distribution found for lxmunidecodel


## <a id='2_web_page'> 2. Webpage </a>

One option direct link (does not work every time)

In [ ]:
#ChromeDriverManager
#driver = webdriver.Chrome( ChromeDriverManager().install() )
#driver.maximize_window()
#url = 'https://resultadoshistorico.onpe.gob.pe/EG2021/EleccionesPresidenciales/RePres/E'
#driver.get( url )
#driver.execute_script( "document.body.style.zoom='100%'" )

Second options, "ONPE Resultado Historico" link and access with clicks. May require time sleep of 10 or more seconds

In [2]:
ChromeDriverManager
driver = webdriver.Chrome( ChromeDriverManager().install() )
driver.maximize_window()
url = 'https://resultadoshistorico.onpe.gob.pe/EG2021/'
driver.get( url )
driver.execute_script( "document.body.style.zoom='100%'" )

presi = driver.find_element_by_xpath('/html/body/onpe-root/onpe-home-onpe/div[1]/div/div/div/div[2]/div[2]/div/div/a/div[2]')
presi.click()
time.sleep(10)



====== WebDriver manager ======
Current google-chrome version is 96.0.4664
Get LATEST chromedriver version for 96.0.4664 google-chrome
Driver [C:\Users\azula\.wdm\drivers\chromedriver\win32\96.0.4664.45\chromedriver.exe] found in cache
<ipython-input-2-5908a993c7a8>:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome( ChromeDriverManager().install() )
<ipython-input-2-5908a993c7a8>:8: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  presi = driver.find_element_by_xpath('/html/body/onpe-root/onpe-home-onpe/div[1]/div/div/div/div[2]/div[2]/div/div/a/div[2]')


## <a id='3_scope'> 3. Scope </a>

In the part we get all the element from the scope and select our element of interest (i.e. Extranjero)

In [3]:
# Get the socpe by element_id
#scope = driver.find_element_by_id( "select_ambito" )
scope = driver.find_element_by_xpath('/html/body/onpe-root/onpe-layout-container/onpe-onpe-epres-re/div[1]/div[3]/div[1]/div[1]/div/div/div/select')

# All selenium objects in scope
scope_options = [ x for x in scope.find_elements_by_tag_name("option") ]
dict_scope_options = { option.text : option for option in scope_options }

# We click on Extranjero
dict_scope_options['EXTRANJERO'].click()

<ipython-input-3-8536e4716e71>:3: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  scope = driver.find_element_by_xpath('/html/body/onpe-root/onpe-layout-container/onpe-onpe-epres-re/div[1]/div[3]/div[1]/div[1]/div/div/div/select')
C:\Users\azula\anaconda3\lib\site-packages\selenium\webdriver\remote\webelement.py:359: UserWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  warnings.warn("find_elements_by_* commands are deprecated. Please use find_elements() instead")


## <a id='4_set_up'> 4. Set Up before loop </a>

In [4]:
# Store all_tables
all_tables = {}

# All selenium objects in continen selection
cnt = Select( driver.find_element_by_xpath( "/html/body/onpe-root/onpe-layout-container/onpe-onpe-epres-re/div[1]/div[3]/div[1]/div[1]/div/div/div[2]/select" ) )

# Get number of total options
num_cont = len( cnt.options )

<ipython-input-4-33c7d3e4b16f>:5: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  cnt = Select( driver.find_element_by_xpath( "/html/body/onpe-root/onpe-layout-container/onpe-onpe-epres-re/div[1]/div[3]/div[1]/div[1]/div/div/div[2]/select" ) )


## <a id='5_loop'> 5. Loop for webscraping </a>

In [7]:
######## PARTE CONTINENTES ########
for cont_idx in range( num_cont ):
    cont = Select(driver.find_element_by_xpath("/html/body/onpe-root/onpe-layout-container/onpe-onpe-epres-re/div[1]/div[3]/div[1]/div[1]/div/div/div[2]/select" ) )
    continent = cont.options[ cont_idx ]
    
    # Get continent name
    cont_name = continent.text
    
    # We select a different continent name
    if cont_name != "--TODOS--" :
        
        # click on continent
        continent.click()
        
######## PARTE PAIS ########

     # Get all elements of country
        pais = Select( driver.find_element_by_xpath("/html/body/onpe-root/onpe-layout-container/onpe-onpe-epres-re/div[1]/div[3]/div[1]/div[1]/div/div/div[3]/select"))
        num_pais = len( pais.options )   
       
        for pais_idx in range( num_pais ):
            pais = Select( driver.find_element_by_xpath("/html/body/onpe-root/onpe-layout-container/onpe-onpe-epres-re/div[1]/div[3]/div[1]/div[1]/div/div/div[3]/select" ) )
            country = pais.options[ pais_idx ]
        
        
        # Get country name
            pais_name = country.text
            if pais_name != "--TODOS--" :
                          
                # click on country
                country.click()

######## PARTE ESTADOS ########

                
                # Get all elements from estado
                est = Select( driver.find_element_by_xpath("/html/body/onpe-root/onpe-layout-container/onpe-onpe-epres-re/div[1]/div[3]/div[1]/div[1]/div/div/div[4]/select" ) )
                num_est = len( est.options )
                
                for est_idx in range( num_est ):
                    
                    # Get again all states 
                    est = Select( driver.find_element_by_xpath("/html/body/onpe-root/onpe-layout-container/onpe-onpe-epres-re/div[1]/div[3]/div[1]/div[1]/div/div/div[4]/select") )
                    state = est.options[ est_idx ]
                    
                    # Get states name
                    est_name = state.text
                    print(est_name)
                    if est_name != "--TODOS--" :
                        
                        # click on state
                        state.click()
                        
######## PARTE CONSOLIDAR ########
                        
                        # Get UBIGEO
                        ubigeo = driver.current_url.split("/")[ -1 ]
                        
                        ## Get table of presidential votes
                        # Get html at this point
                        # No id for the table wrap, so full xpath 
                        # is used instead
                        table_path = driver.find_element_by_xpath( "/html/body/onpe-root/onpe-layout-container/onpe-onpe-epres-re/div[1]/div[4]/div[1]/div[3]/div" )
                        table_html = table_path.get_attribute( 'innerHTML' )
                        # Read the table using pandas
                        table = pd.read_html( table_html )
                        
                        # Cleaning tables
                        row_new_columns = table[ 0 ].iloc[ 0 , 2: ]
                        clean_columns = row_new_columns \
                                              .str.replace( " ", "_") \
                                              .str.lower().str.replace( "%", "share_") \
                                              .apply( lambda x : unidecode.unidecode( x ) ) \
                                              .tolist()
                        
                        # Selecting specific columns
                        table_clean = table[0].iloc[ 1:, 2: ].copy()
                        
                        # rename columns
                        table_clean.columns = clean_columns
                        
                        # New values to columns 
                        table_clean[ 'continent' ] = cont_name
                        table_clean[ 'country' ]   = pais_name
                        table_clean[ 'estate' ]    = est_name
                        table_clean[ 'ubigeo' ]    = ubigeo
                        table_clean[ 'link' ]      = driver.current_url
                        
                        # store tables
                        all_tables[ ubigeo ] = table_clean                       

<ipython-input-7-1d7d495d847e>:3: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  cont = Select(driver.find_element_by_xpath("/html/body/onpe-root/onpe-layout-container/onpe-onpe-epres-re/div[1]/div[3]/div[1]/div[1]/div/div/div[2]/select" ) )
<ipython-input-7-1d7d495d847e>:18: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  pais = Select( driver.find_element_by_xpath("/html/body/onpe-root/onpe-layout-container/onpe-onpe-epres-re/div[1]/div[3]/div[1]/div[1]/div/div/div[3]/select"))
<ipython-input-7-1d7d495d847e>:22: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  pais = Select( driver.find_element_by_xpath("/html/body/onpe-root/onpe-layout-container/onpe-onpe-epres-re/div[1]/div[3]/div[1]/div[1]/div/div/div[3]/select" ) )
<ipython-input-7-1d7d495d847e>:37: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_el

--TODOS--
ARGEL


<ipython-input-7-1d7d495d847e>:63: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  table_path = driver.find_element_by_xpath( "/html/body/onpe-root/onpe-layout-container/onpe-onpe-epres-re/div[1]/div[4]/div[1]/div[3]/div" )
<ipython-input-7-1d7d495d847e>:22: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  pais = Select( driver.find_element_by_xpath("/html/body/onpe-root/onpe-layout-container/onpe-onpe-epres-re/div[1]/div[3]/div[1]/div[1]/div/div/div[3]/select" ) )
<ipython-input-7-1d7d495d847e>:37: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  est = Select( driver.find_element_by_xpath("/html/body/onpe-root/onpe-layout-container/onpe-onpe-epres-re/div[1]/div[3]/div[1]/div[1]/div/div/div[4]/select" ) )
<ipython-input-7-1d7d495d847e>:43: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  

--TODOS--
RABAT


<ipython-input-7-1d7d495d847e>:63: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  table_path = driver.find_element_by_xpath( "/html/body/onpe-root/onpe-layout-container/onpe-onpe-epres-re/div[1]/div[4]/div[1]/div[3]/div" )
<ipython-input-7-1d7d495d847e>:22: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  pais = Select( driver.find_element_by_xpath("/html/body/onpe-root/onpe-layout-container/onpe-onpe-epres-re/div[1]/div[3]/div[1]/div[1]/div/div/div[3]/select" ) )
<ipython-input-7-1d7d495d847e>:37: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  est = Select( driver.find_element_by_xpath("/html/body/onpe-root/onpe-layout-container/onpe-onpe-epres-re/div[1]/div[3]/div[1]/div[1]/div/div/div[4]/select" ) )
<ipython-input-7-1d7d495d847e>:43: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  

...

PERTH
SIDNEY


<ipython-input-7-1d7d495d847e>:43: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  est = Select( driver.find_element_by_xpath("/html/body/onpe-root/onpe-layout-container/onpe-onpe-epres-re/div[1]/div[3]/div[1]/div[1]/div/div/div[4]/select") )
<ipython-input-7-1d7d495d847e>:63: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  table_path = driver.find_element_by_xpath( "/html/body/onpe-root/onpe-layout-container/onpe-onpe-epres-re/div[1]/div[4]/div[1]/div[3]/div" )
<ipython-input-7-1d7d495d847e>:22: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  pais = Select( driver.find_element_by_xpath("/html/body/onpe-root/onpe-layout-container/onpe-onpe-epres-re/div[1]/div[3]/div[1]/div[1]/div/div/div[3]/select" ) )
<ipython-input-7-1d7d495d847e>:37: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  e

--TODOS--
AUCKLAND


<ipython-input-7-1d7d495d847e>:63: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  table_path = driver.find_element_by_xpath( "/html/body/onpe-root/onpe-layout-container/onpe-onpe-epres-re/div[1]/div[4]/div[1]/div[3]/div" )
<ipython-input-7-1d7d495d847e>:43: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  est = Select( driver.find_element_by_xpath("/html/body/onpe-root/onpe-layout-container/onpe-onpe-epres-re/div[1]/div[3]/div[1]/div[1]/div/div/div[4]/select") )
<ipython-input-7-1d7d495d847e>:63: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  table_path = driver.find_element_by_xpath( "/html/body/onpe-root/onpe-layout-container/onpe-onpe-epres-re/div[1]/div[4]/div[1]/div[3]/div" )


WELLINGTON


## <a id='6_final_data'> 6. Data processing and storing </a>

In [8]:
final_data = pd.concat( all_tables.values() ).reset_index( drop = True )
final_data = final_data.rename( columns={'organizaciones_politicas': ' political_parties',
                                         'share_validos': 'share_valid',
                                         'share_emitidos': 'share_emitied'})
final_data.to_excel( r'../_data_results/group_5_ass_4.xlsx' , index = False )
final_data

,political_parties,total,share_valid,share_emitied,continent,country,estate,ubigeo,link
0,PARTIDO NACIONALISTA PERUANO,0,0.000%,0.000%,AFRICA,ARGELIA,ARGEL,910101,https://resultadoshistorico.onpe.gob.pe/EG2021...
1,"EL FRENTE AMPLIO POR JUSTICIA, VIDA Y LIBERTAD",0,0.000%,0.000%,AFRICA,ARGELIA,ARGEL,910101,https://resultadoshistorico.onpe.gob.pe/EG2021...
2,PARTIDO MORADO,0,0.000%,0.000%,AFRICA,ARGELIA,ARGEL,910101,https://resultadoshistorico.onpe.gob.pe/EG2021...
3,PERU PATRIA SEGURA,0,0.000%,0.000%,AFRICA,ARGELIA,ARGEL,910101,https://resultadoshistorico.onpe.gob.pe/EG2021...
4,VICTORIA NACIONAL,0,0.000%,0.000%,AFRICA,ARGELIA,ARGEL,910101,https://resultadoshistorico.onpe.gob.pe/EG2021...
...,...,...,...,...,...,...,...,...,...
4658,ALIANZA PARA EL PROGRESO,0,0.000%,0.000%,OCEANIA,NUEVA ZELANDA,WELLINGTON,950202,https://resultadoshistorico.onpe.gob.pe/EG2021...
4659,TOTAL DE VOTOS VÁLIDOS,34,100.000%,97.143%,OCEANIA,NUEVA ZELANDA,WELLINGTON,950202,https://resultadoshistorico.onpe.gob.pe/EG2021...
4660,VOTOS EN BLANCO,0,NaN,0.000%,OCEANIA,NUEVA ZELANDA,WELLINGTON,950202,https://resultadoshistorico.onpe.gob.pe/EG2021...
4661,VOTOS NULOS,1,NaN,2.857%,OCEANIA,NUEVA ZELANDA,WELLINGTON,950202,https://resultadoshistorico.onpe.gob.pe/EG2021...
